# Prototype SPHINCS+

## Enunciado do Problema

Neste trabalho pretende-se implementar em Sagemath de algumas dos candidatos a “standartização” ao concurso NIST Post-Quantum Cryptography na categoria de esquemas de assinatura digital. Ver também a directoria com a documentação. Construa
- Um protótipo Sagemath do algoritmo Sphincs+.

## Descrição do Problema


## Abordagem

## Código

Segue-se o código desenvolvido para este protótipo do algoritmo **SPHINCS+**

### Definição dos parâmetros

Os parâmetros foram definidos para a variante **SPHINCS+-128s**

In [12]:
n = 16    # The security parameter in bytes
w = 16    # The Winternitz parameter
h = 63    # The height of the hypertree
d = 7     # The number of layers in the hypertree
k = 14    # The number of trees in FORS
a = 12    # The number of leaves of a FORS tree

### Math Functions needed

In [24]:
import math


def lg(x: int) -> float:
    return math.log(x, 2)

def ceil(x: float) -> int:
    return math.ceil(x)

def floor(x: float) -> int:
    return math.floor(x)

### Function toByte

In [33]:
def toByte(x: int, y: int):
    """
    x, y non-negative integers
    x - number to be converted
    y - number of bytes in the output
    Z = toByte(x, y) is the y-byte string representation of x
    """

    return x.to_bytes(y, 'big')


x = 255
y = 2

result = toByte(x, y)
print(result)  # Output: b'\x00\xff'


b'\x00\xff'


### Function base_w

In [22]:
from typing import List

def base_w(X: str, w: int, out_len: int) -> List[int]:
    """
    Input: len_X-byte string X, int w, output length out_len
    Output: out_len int array basew
    """
    _in = 0
    out = 0
    total = 0
    bits = 0
    basew = []

    for consumed in range(out_len):
        if bits == 0:
            total = X[_in]
            _in += 1
            bits += 8
        bits -= int(lg(w))
        basew.append((total >> bits) & (w - 1))
        out += 1

    return basew

X = b'\x12\x34'
w = 16
out_len = 4

result = base_w(X, w, out_len)
print(result)  # Output: [1, 2, 3, 4]

# TODO: Add Assert verifications in all functions


[1, 2, 3, 4]


### Definição da estrutura do ADRS

In [ ]:
class ADRS:
    # Types
    TYPE_WOTS = 0
    TYPE_WOTSPK = 1
    TYPE_HASHTREE = 2
    TYPE_FORSTREE = 3
    TYPE_FORSPK = 4

    def __init__(self):
        self.layer_address = 0
        self.tree_address = 0
        self.type = 0

        # Differs from type to type
        self.word1 = 0
        self.word2 = 0
        self.word3 = 0

    # Setters
    def set_layer_address(self, layer_address: int):
        self.layer_address = layer_address

    def set_tree_address(self, tree_address: int):
        self.tree_address = tree_address

    def set_type(self, type: int):
        self.type = type

    def set_word1(self, word1: int):
        self.word1 = word1

    def set_word2(self, word2: int):
        self.word2 = word2

    def set_word3(self, word3: int):
        self.word3 = word3

    # Specific setters for each type
    def set_hash_address(self, hash_address: int):
        self.set_word1(hash_address)

    def to_bytes(self) -> bytes:
        result = b''
        result += toByte(self.layer_address, 4)
        result += toByte(self.tree_address, 12)
        result += toByte(self.type, 4)
        result += toByte(self.word1, 4)
        result += toByte(self.word2, 4)
        result += toByte(self.word3, 4)

        return result

### F, H, T Functions (Tweakable Hash Functions)

For the simple variant, we instead define the tweakable F functions as
```math
F(PK.seed, ADRS, M1) = SHAKE256(PK.seed || ADRS|| M1, 8n),
H(PK.seed, ADRS, M1||M2) = SHAKE256(PK.seed || ADRS || M1 || M2, 8n),
T(PK.seed, ADRS, M) = SHAKE256(PK.seed || ADRS || M, 8n)
```

In [34]:
import hashlib


def T(pk_seed: bytes, adrs: ADRS, m: bytes) -> bytes:
    """
    Input: pk_seed, adrs, m
    Output: m F
    """
    shake = hashlib.shake_256()

    shake.update(pk_seed)
    shake.update(adrs.to_bytes())
    shake.update(m)

    return shake.digest(8 * n)


def F(pk_seed: bytes, adrs: ADRS, m: bytes) -> bytes:
    """
    Input: pk_seed, adrs, m
    Output: m F
    """
    shake = hashlib.shake_256()

    shake.update(pk_seed)
    shake.update(adrs.to_bytes())
    shake.update(m)

    return shake.digest(8 * n)


def H(pk_seed: bytes, adrs: ADRS, m: bytes) -> bytes:
    """
    Input: pk_seed, adrs, m
    Output: m F
    """
    shake = hashlib.shake_256()

    shake.update(pk_seed)
    shake.update(adrs.to_bytes())
    shake.update(m)

    return shake.digest(8 * n)

### PRF, PRFmsg, Hmsg Functions

**PRF** function is used for pseudorandom key generation.

**PRFmsg** function is used to generate randomness for the message compression.

**Hmsg** function is used to compress the message to be signed.

In [38]:
def PRF(seed: bytes, adrs: ADRS) -> bytes:
    """
    Input: seed, adrs
    Output: F
    """
    shake = hashlib.shake_256()

    shake.update(seed)
    shake.update(adrs.to_bytes())

    return shake.digest(8 * n)


def PRF_msg(seed: bytes, adrs: ADRS, m: bytes) -> bytes:
    """
    Input: seed, adrs, m
    Output: F
    """
    shake = hashlib.shake_256()

    shake.update(seed)
    shake.update(adrs.to_bytes())
    shake.update(m)

    return shake.digest(8 * n)


def H_msg(msg: bytes) -> bytes:
    """
    Input: msg
    Output: F
    """
    shake = hashlib.shake_256()

    shake.update(msg)

    return shake.digest(8 * m)

### WOTS+ One Time Signatures

**WOTS+** é um esquema OTS (One-Time Signature).

De notar que apesar das chaves privadas poderem assinar quaisquer mensagens, cada chave privada **NÃO DEVE SER USADA PARA ASSINAR MAIS DE QUE UMA MENSAGEM**.

#### Parâmetros

In [35]:
# n already defined above
# w already defined above (can be chosen from the set {4, 16, 256})

len1 = ceil((8 * n) / lg(w))
len2 = floor(lg(len1 * (w - 1)) / lg(w)) + 1

_len = len1 + len2

#### Chain Function

In [36]:
from typing import Optional


def chain(X:str, i: int, s: int, pk_seed, adrs: ADRS) -> Optional[bytes]:
    """
    Input: Input string X, start index i, number of steps s, public seed PK.seed, address ADRS
    Output: value of F iterated s times on X
    """
    if s == 0:
        return bytes(X)

    if (i + s) > (w - 1):
        return None

    tmp = chain(X, i, s - 1, pk_seed, adrs)

    adrs.set_hash_address(i + s - 1)

    tmp = F(pk_seed, adrs, tmp)

    return tmp

####  wost_SKgen Function

In [ ]:
def wots_pk_gen(seed: bytes, adrs: ADRS, wots_sk: bytes) -> Optional[bytes]:
    """
    Input: secret seed, address ADRS, address ADRS
    Output: WOTS+ private key PK
    """
    pass


#### wots_PKgen Function

## Testes/Exemplos

...